# C. クリーンアップ - リソースの削除

## 概要

このノートブックでは、ハンズオンで作成したリソースをクリーンアップします:

1. **インデクサの停止と削除** (Pattern B)
2. **スキルセットの削除** (Pattern B)
3. **データソースの削除** (Pattern B)
4. **インデックスの削除** (Pattern A & B)
5. **Delta トークンファイルの削除** (Pattern A)

## 注意事項

⚠️ **警告**: このノートブックを実行すると、すべてのデータとリソースが削除されます。

- インデックス内のすべてのドキュメントが削除されます
- インデクサとスキルセットの設定が失われます
- 復元はできません

本番環境では慎重に実行してください。

## 1. 環境の初期化

In [ ]:
import os
from pathlib import Path
from dotenv import load_dotenv

from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient, SearchIndexerClient
from azure.core.exceptions import ResourceNotFoundError

# 環境変数の読み込み
env_path = Path("../.env")
if env_path.exists():
    load_dotenv(env_path)
    print("✅ .env ファイルを読み込みました")
else:
    print("⚠️ .env ファイルが見つかりません")

# 必須環境変数の確認
required_vars = ["AZURE_SEARCH_ENDPOINT", "AZURE_SEARCH_API_KEY"]
missing_vars = [var for var in required_vars if not os.getenv(var)]
if missing_vars:
    raise ValueError(f"環境変数が設定されていません: {', '.join(missing_vars)}")

print("\n✅ 環境変数の確認完了")

## 2. 設定とクライアントの初期化

In [ ]:
# Azure AI Search 設定
search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
search_key = os.getenv("AZURE_SEARCH_API_KEY")
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME", "sp-docs")

# Pattern B のリソース名
index_name_indexer = os.getenv("AZURE_SEARCH_INDEX_NAME", "sp-docs-indexer")
datasource_name = f"{index_name_indexer}-datasource"
skillset_name = f"{index_name_indexer}-skillset"
indexer_name = f"{index_name_indexer}-indexer"

# 認証情報
credential = AzureKeyCredential(search_key)

# インデックス管理クライアント
index_client = SearchIndexClient(
    endpoint=search_endpoint,
    credential=credential
)

# インデクサ管理クライアント
indexer_client = SearchIndexerClient(
    endpoint=search_endpoint,
    credential=credential
)

print(f"✅ クライアント作成完了")
print(f"   エンドポイント: {search_endpoint}")
print(f"\n削除対象リソース:")
print(f"   Pattern A インデックス: {index_name}")
print(f"   Pattern B インデックス: {index_name_indexer}")
print(f"   インデクサ: {indexer_name}")
print(f"   スキルセット: {skillset_name}")
print(f"   データソース: {datasource_name}")

## 3. 現在のリソース一覧

In [ ]:
print("📋 現在のリソース一覧\n")
print("="*80)

# インデックス一覧
print("\n📚 インデックス:")
try:
    indexes = index_client.list_indexes()
    index_count = 0
    for idx in indexes:
        print(f"  - {idx.name}")
        index_count += 1
    if index_count == 0:
        print("  (なし)")
except Exception as e:
    print(f"  ⚠️ エラー: {e}")

# インデクサ一覧
print("\n🔄 インデクサ:")
try:
    indexers = indexer_client.list_indexers()
    indexer_count = 0
    for indexer in indexers:
        print(f"  - {indexer.name}")
        indexer_count += 1
    if indexer_count == 0:
        print("  (なし)")
except Exception as e:
    print(f"  ⚠️ エラー: {e}")

# スキルセット一覧
print("\n🎯 スキルセット:")
try:
    skillsets = indexer_client.list_skillsets()
    skillset_count = 0
    for skillset in skillsets:
        print(f"  - {skillset.name}")
        skillset_count += 1
    if skillset_count == 0:
        print("  (なし)")
except Exception as e:
    print(f"  ⚠️ エラー: {e}")

# データソース一覧
print("\n💾 データソース:")
try:
    datasources = indexer_client.list_data_source_connections()
    datasource_count = 0
    for ds in datasources:
        print(f"  - {ds.name}")
        datasource_count += 1
    if datasource_count == 0:
        print("  (なし)")
except Exception as e:
    print(f"  ⚠️ エラー: {e}")

print("\n" + "="*80)

## 4. インデクサの停止と削除 (Pattern B)

まずインデクサを停止してから削除します。

In [ ]:
print(f"🔄 インデクサ '{indexer_name}' の削除\n")

try:
    # インデクサの存在確認
    indexer = indexer_client.get_indexer(indexer_name)
    print(f"✅ インデクサが見つかりました: {indexer_name}")
    
    # インデクサの削除
    indexer_client.delete_indexer(indexer_name)
    print(f"✅ インデクサを削除しました: {indexer_name}")
    
except ResourceNotFoundError:
    print(f"ℹ️ インデクサが見つかりません: {indexer_name}")
except Exception as e:
    print(f"⚠️ インデクサ削除エラー: {e}")

## 5. スキルセットの削除 (Pattern B)

In [ ]:
print(f"🎯 スキルセット '{skillset_name}' の削除\n")

try:
    # スキルセットの存在確認
    skillset = indexer_client.get_skillset(skillset_name)
    print(f"✅ スキルセットが見つかりました: {skillset_name}")
    
    # スキルセットの削除
    indexer_client.delete_skillset(skillset_name)
    print(f"✅ スキルセットを削除しました: {skillset_name}")
    
except ResourceNotFoundError:
    print(f"ℹ️ スキルセットが見つかりません: {skillset_name}")
except Exception as e:
    print(f"⚠️ スキルセット削除エラー: {e}")

## 6. データソースの削除 (Pattern B)

In [ ]:
print(f"💾 データソース '{datasource_name}' の削除\n")

try:
    # データソースの存在確認
    datasource = indexer_client.get_data_source_connection(datasource_name)
    print(f"✅ データソースが見つかりました: {datasource_name}")
    
    # データソースの削除
    indexer_client.delete_data_source_connection(datasource_name)
    print(f"✅ データソースを削除しました: {datasource_name}")
    
except ResourceNotFoundError:
    print(f"ℹ️ データソースが見つかりません: {datasource_name}")
except Exception as e:
    print(f"⚠️ データソース削除エラー: {e}")

## 7. インデックスの削除

Pattern A と Pattern B のインデックスを削除します。

⚠️ **警告**: インデックス内のすべてのドキュメントが削除されます。

In [ ]:
print("📚 インデックスの削除\n")

# Pattern A のインデックス
print(f"Pattern A インデックス '{index_name}':")
try:
    index = index_client.get_index(index_name)
    print(f"  ✅ インデックスが見つかりました")
    
    index_client.delete_index(index_name)
    print(f"  ✅ インデックスを削除しました: {index_name}")
    
except ResourceNotFoundError:
    print(f"  ℹ️ インデックスが見つかりません: {index_name}")
except Exception as e:
    print(f"  ⚠️ インデックス削除エラー: {e}")

# Pattern B のインデックス（Pattern A と異なる場合のみ）
if index_name != index_name_indexer:
    print(f"\nPattern B インデックス '{index_name_indexer}':")
    try:
        index = index_client.get_index(index_name_indexer)
        print(f"  ✅ インデックスが見つかりました")
        
        index_client.delete_index(index_name_indexer)
        print(f"  ✅ インデックスを削除しました: {index_name_indexer}")
        
    except ResourceNotFoundError:
        print(f"  ℹ️ インデックスが見つかりません: {index_name_indexer}")
    except Exception as e:
        print(f"  ⚠️ インデックス削除エラー: {e}")

## 8. Delta トークンファイルの削除 (Pattern A)

Pattern A で使用した差分同期のステートファイルを削除します。

In [ ]:
print("📄 Delta トークンファイルの削除\n")

delta_token_file = os.getenv("DELTA_TOKEN_FILE", ".delta_token.json")
delta_token_path = Path("..") / delta_token_file

try:
    if delta_token_path.exists():
        delta_token_path.unlink()
        print(f"✅ Delta トークンファイルを削除しました: {delta_token_path}")
    else:
        print(f"ℹ️ Delta トークンファイルが見つかりません: {delta_token_path}")
except Exception as e:
    print(f"⚠️ ファイル削除エラー: {e}")

## 9. 削除の確認

In [ ]:
print("🔍 削除の確認\n")
print("="*80)

all_deleted = True

# インデックスの確認
print("\n📚 インデックス:")
for idx_name in [index_name, index_name_indexer]:
    if idx_name:
        try:
            index_client.get_index(idx_name)
            print(f"  ⚠️ まだ存在します: {idx_name}")
            all_deleted = False
        except ResourceNotFoundError:
            print(f"  ✅ 削除済み: {idx_name}")
        except Exception as e:
            print(f"  ⚠️ 確認エラー: {e}")

# インデクサの確認
print("\n🔄 インデクサ:")
try:
    indexer_client.get_indexer(indexer_name)
    print(f"  ⚠️ まだ存在します: {indexer_name}")
    all_deleted = False
except ResourceNotFoundError:
    print(f"  ✅ 削除済み: {indexer_name}")
except Exception as e:
    print(f"  ⚠️ 確認エラー: {e}")

# スキルセットの確認
print("\n🎯 スキルセット:")
try:
    indexer_client.get_skillset(skillset_name)
    print(f"  ⚠️ まだ存在します: {skillset_name}")
    all_deleted = False
except ResourceNotFoundError:
    print(f"  ✅ 削除済み: {skillset_name}")
except Exception as e:
    print(f"  ⚠️ 確認エラー: {e}")

# データソースの確認
print("\n💾 データソース:")
try:
    indexer_client.get_data_source_connection(datasource_name)
    print(f"  ⚠️ まだ存在します: {datasource_name}")
    all_deleted = False
except ResourceNotFoundError:
    print(f"  ✅ 削除済み: {datasource_name}")
except Exception as e:
    print(f"  ⚠️ 確認エラー: {e}")

print("\n" + "="*80)

if all_deleted:
    print("\n✅ すべてのリソースが正常に削除されました")
else:
    print("\n⚠️ 一部のリソースがまだ残っています")
    print("   必要に応じて Azure Portal から手動で削除してください")

## 10. Azure リソースの確認

このノートブックでは、Azure AI Search サービス内のリソース（インデックス、インデクサなど）のみを削除しました。

以下のAzureリソースは削除されていません:

### 削除されていないリソース

1. **Azure AI Search サービス** - 課金が継続します
2. **Azure OpenAI サービス** - 課金が継続します
3. **Azure Document Intelligence** - 課金が継続します
4. **Azure AD アプリ登録** - 認証情報が残ります
5. **SharePoint データ** - 元データはそのまま残ります

### 完全なクリーンアップ

課金を停止するには、Azure Portal から以下を削除してください:

```
1. Azure Portal にログイン
2. リソースグループを開く
3. 不要なリソースを選択
4. 「削除」をクリック
```

### Azure AD アプリ登録の削除

```
1. Azure Portal > Microsoft Entra ID
2. アプリの登録
3. 対象のアプリを選択
4. 「削除」をクリック
```

## まとめ

このノートブックでは、ハンズオンで作成したリソースをクリーンアップしました:

1. ✅ **インデクサの削除**: 自動取り込みパイプラインの停止
2. ✅ **スキルセットの削除**: AI スキルの削除
3. ✅ **データソースの削除**: SharePoint 接続の削除
4. ✅ **インデックスの削除**: すべてのドキュメントの削除
5. ✅ **Delta トークンの削除**: 差分同期のステート削除

### 削除されたもの

- ✅ Azure AI Search のインデックスとドキュメント
- ✅ インデクサとスキルセット
- ✅ データソース接続
- ✅ Delta 同期のステートファイル

### 残っているもの（手動削除が必要）

- ⚠️ Azure AI Search サービス本体
- ⚠️ Azure OpenAI サービス
- ⚠️ Azure Document Intelligence
- ⚠️ Azure AD アプリ登録
- ⚠️ SharePoint のドキュメント

### 再開する場合

ハンズオンを再開する場合は:

1. **Pattern A**: `A1_graph_ingest_setup.ipynb` から開始
2. **Pattern B**: `B1_sp_indexer_setup.ipynb` から開始

### 参考リソース

- [Azure リソースの管理](https://learn.microsoft.com/azure/azure-resource-manager/management/manage-resources-portal)
- [Azure AI Search の価格](https://azure.microsoft.com/pricing/details/search/)
- [コスト管理のベストプラクティス](https://learn.microsoft.com/azure/cost-management-billing/costs/cost-mgt-best-practices)